In [1]:
import keras
import os
import pandas as pd
import numpy as np
import tensorflow as tf
import cv2
import sys

from os import listdir
from os.path import isfile, join
from sklearn.model_selection import train_test_split
from keras.layers import Input
from keras.models import Model
from keras.optimizers import Adam
from Template_LSTM import LSTM_TEMPLATE as LSTM
from Template_ResNet import ResNet_TEMPLATE as ResNet
from Template_MLP import MLP
from keras.callbacks import TensorBoard
from datetime import datetime

Using TensorFlow backend.


In [2]:
# LSTM ParamTest
name_param = "LSTMtest1"
print("name_param = " + name_param)
name_modele = "LSTM"
print("name_modele = " + name_modele)
batch_size = 512
print("batch_size = " + str(batch_size))
epochs = 70
print("epochs = " + str(epochs))
lera = 0.001
print("lera = " + str(lera))
activation = ""
print("activation = " + str(activation))
nb_layer = 5
print("nb_layer = " + str(nb_layer))
nb_filtre = 32
print("nb_filtre = " + str(nb_filtre))
nb_dropout_flag = 1
print("nb_dropout_flag = " + str(nb_dropout_flag))
nb_dropout_value = 0.2
print("nb_dropout_value = " + str(nb_dropout_value))

name_param = LSTMtest1
name_modele = LSTM
batch_size = 512
epochs = 70
lera = 0.001
activation = 
nb_layer = 5
nb_filtre = 32
nb_dropout_flag = 1
nb_dropout_value = 0.2


In [3]:
# ResNet ParamTest
name_param = "ResNettest1"
print("name_param = " + name_param)
name_modele = "ResNet"
print("name_modele = " + name_modele)
batch_size = 512
print("batch_size = " + str(batch_size))
epochs = 70
print("epochs = " + str(epochs))
lera = 0.001
print("lera = " + str(lera))
activation = "relu"
print("activation = " + activation)
nb_layer = 3
print("nb_layer = " + str(nb_layer))
nb_filtre = ""
print("nb_filtre = " + str(nb_filtre))
nb_dropout_flag = 1
print("nb_dropout_flag = " + str(nb_dropout_flag))
nb_dropout_value = 0.2
print("nb_dropout_value = " + str(nb_dropout_value))

name_param = ResNettest1
name_modele = ResNet
batch_size = 512
epochs = 70
lera = 0.001
activation = relu
nb_layer = 3
nb_filtre = 
nb_dropout_flag = 1
nb_dropout_value = 0.2


In [4]:
# Preprocessing
train_csv = pd.read_csv('../Data/data/train.csv')

filenames = ['../Data/data/train/' + fname for fname in train_csv['id'].tolist()]
labels = train_csv['has_cactus'].tolist()

train = []
for file_name in filenames:
    img = cv2.imread(file_name)
    img = img.reshape(32*32*3,)
    img = img/255
    train.append(img)

x_train, x_test, y_train, y_test = train_test_split(train,
                                                    labels,
                                                    train_size=0.9)


x_train = np.array(x_train)
x_train = np.reshape(x_train, (-1, 32, 32, 3))
print(x_train.shape)

x_test = np.array(x_test)
x_test = np.reshape(x_test, (-1, 32, 32, 3))
print(x_test.shape)


inputs = Input(shape=(32, 32, 3))
print(inputs)

y_train = keras.utils.to_categorical(y_train, 2)
y_test = keras.utils.to_categorical(y_test, 2)

/anaconda2/envs/tensorflowlatest/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


(15750, 32, 32, 3)
(1750, 32, 32, 3)
Tensor("input_1:0", shape=(?, 32, 32, 3), dtype=float32)


In [5]:
# Preaparation modèle
if name_modele == "LSTM":
    print(name_modele + " " + name_param)
    outputs = LSTM(inputs, nb_filtre, nb_layer, nb_dropout_flag, nb_dropout_value)
if name_modele == "ResNet":
    print(name_modele + " " + name_param)
    outputs = ResNet(nb_layer, inputs, activation, nb_dropout_flag, nb_dropout_value)
if name_modele == "MLP":
    outputs = MLP(inputs, activation, nb_layer, nb_filtre)

## Run model
model = Model(inputs=inputs, outputs=outputs)

if name_modele == 'MLP':
    model.compile(loss='binary_crossentropy',
                  optimizer=Adam(lr=float(lera)),
                  metrics=['accuracy'])
else:
    model.compile(loss='categorical_crossentropy',
                  optimizer=Adam(lr=float(lera)),
                  metrics=['accuracy'])

model.summary()

ResNet ResNettest1
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 32, 32, 16)   448         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 32, 32, 16)   64          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 32, 32, 16)   0           batch_normalization_1[0][0]      
__________________________________________________________________________________________

In [6]:
# Prepare model model saving directory.
save_dir = os.path.join(os.getcwd(), 'res_logs')
date = datetime.today()
year = date.strftime("%Y")
month = date.strftime("%m")
day = date.strftime("%d")
hour = date.strftime("%H")
minute = date.strftime("%M")
model_name = "{}{}{}{}{}{}{}_es{}_lr{}_bs{}_{}_ly{}_nf{}" \
    .format(name_param, name_modele, year, month, day, hour, minute, epochs, lera, batch_size, activation, nb_layer, nb_filtre)
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
filepath = os.path.join(save_dir, model_name)
callbacks = TensorBoard(log_dir=filepath)

In [7]:
# Run training
model.fit(x_train, y_train,
          batch_size=int(batch_size),
          epochs=int(1), #int(epochs)
          validation_data=(x_test, y_test),
          callbacks=[callbacks])

Train on 15750 samples, validate on 1750 samples
Epoch 1/1
15750/15750 [==============================] - 545s 35ms/step - loss: 0.7541 - acc: 0.9369 - val_loss: 0.8231 - val_acc: 0.8954


In [8]:
# predicted class
filenames_test = ['../Data/data/test/' + f for f in listdir('../Data/data/test/') if isfile(join('../Data/data/test/', f))]
print(len(filenames_test))
test = []
i = 0
for file_name in filenames_test:
    img = cv2.imread(file_name)
    # print("--------------------------------------------------------------------------------------------------"+str(i)+str(img))
    # i=i+1
    img = img.reshape(32 * 32 * 3, )
    img = img / 255
    test.append(img)
test = np.array(test)
test = np.reshape(test, (-1, 32, 32, 3))
pred1 = model.predict(test)
predf = pred1.argmax(axis=-1)
print(predf)
# np.savetxt("predict.csv", np.c_[filenames_test,predf], delimiter=",")
df = pd.DataFrame({"id": filenames_test, "has_cactus": predf})
df.to_csv("./prediction/predict_{}_{}.csv".format(name_modele, name_param), index=False)

4000
[1 1 1 ... 0 1 1]


In [9]:
import keras

from keras.layers import Conv2D, BatchNormalization, Activation, Flatten, Dense, concatenate
from keras.layers import multiply, add
from keras.layers import Input

def LSTM_TEMPLATE (inputs, nb_filtre, nb_layer, dropout_flag, dropout_value):
    nb_filtre = int(nb_filtre)
    nb_filtre_b = nb_filtre*2

    print(nb_filtre)
    print(nb_filtre_b)

    x = Conv2D(nb_filtre,
               kernel_size=3,
               strides=1,
               padding='same')(inputs)
    x = BatchNormalization()(x)
    for i in range(int(nb_layer)):
        # Layer 1
        C = Conv2D(nb_filtre_b,
                   kernel_size=3,
                   strides=1,
                   padding='same')(x)
        C = BatchNormalization()(C)

        H = Conv2D(nb_filtre,
                   kernel_size=3,
                   strides=1,
                   padding='same')(x)
        H = BatchNormalization()(H)

        Hx = concatenate([H, x])
        F = Activation('sigmoid')(Hx)

        I = Activation('sigmoid')(Hx)
        L = Activation('tanh')(Hx)
        IL = multiply([I, L])

        C = multiply([C, F])
        C = add([C, IL])

        C = Activation('tanh')(C)
        O = Activation('sigmoid')(Hx)
        x = multiply([C, O])

        nb_filtre_b = nb_filtre_b + nb_filtre

    outputs = Conv2D(nb_filtre,
                      kernel_size=3,
                      strides=1,
                      padding='same')(x)

    x = BatchNormalization()(outputs)

    if dropout_flag == 1 or dropout_flag == 4 or dropout_flag == 5 or dropout_flag == 7:
        x = Dropout(dropout_value)(x)

    x = keras.layers.add([x, outputs])

    y = Flatten()(x)

    if dropout_flag == 2 or dropout_flag == 4 or dropout_flag == 6 or dropout_flag == 7:
        y = Dropout(dropout_value)(y)

    outputs = Dense(2, activation='softmax')(y)

    if dropout_flag == 3 or dropout_flag == 5 or dropout_flag == 6 or dropout_flag == 7:
        outputs = Dropout(dropout_value)(outputs)

    return outputs


In [10]:
import keras

from keras.layers import Input, Conv2D, BatchNormalization, Activation, AveragePooling2D, Flatten, Dense, Dropout
from keras.regularizers import l2

# -----------------
#           |
# Model     |  n
#           |v1(v2)
# -----------------
# ResNet20  | 3 (2)
# ResNet32  | 5(NA)
# ResNet44  | 7(NA)
# ResNet56  | 9 (6)
# ResNet110 |18(12)
# ResNet164 |27(18)
# ResNet1001| (111)
# -----------------

def resnet_layer(inputs,
                 num_filters=16,
                 kernel_size=3,
                 strides=1,
                 activation='relu',
                 batch_normalization=True,
                 conv_first=True):

    conv = Conv2D(num_filters,
                  kernel_size=kernel_size,
                  strides=strides,
                  padding='same',
                  kernel_initializer='he_normal',
                  kernel_regularizer=l2(1e-4))

    x = inputs
    if conv_first:
        x = conv(x)
        if batch_normalization:
            x = BatchNormalization()(x)
        if activation is not None:
            x = Activation(activation)(x)
    else:
        if batch_normalization:
            x = BatchNormalization()(x)
        if activation is not None:
            x = Activation(activation)(x)
        x = conv(x)
    return x

def ResNet_TEMPLATE(n, inputs, activation, dropout_flag, dropout_value):
    num_filters_in = 16
    num_classes = 2
    # Start model definition.

    x = resnet_layer(inputs=inputs,
                     num_filters=num_filters_in,
                     conv_first=True)

    # Instantiate the stack of residual units
    for stage in range(3):
        for res_block in range(int(n)):
            batch_normalization = True
            strides = 1
            if stage == 0:
                num_filters_out = num_filters_in * 4
                if res_block == 0:  # first layer and first stage
                    activation = None
                    batch_normalization = False
            else:
                num_filters_out = num_filters_in * 2
                if res_block == 0:  # first layer but not first stage
                    strides = 2    # downsample

            # bottleneck residual unit
            y = resnet_layer(inputs=x,
                             num_filters=num_filters_in,
                             kernel_size=1,
                             strides=strides,
                             activation=activation,
                             batch_normalization=batch_normalization,
                             conv_first=False)
            y = resnet_layer(inputs=y,
                             num_filters=num_filters_in,
                             conv_first=False)
            y = resnet_layer(inputs=y,
                             num_filters=num_filters_out,
                             kernel_size=1,
                             conv_first=False)
            if res_block == 0:
                # linear projection residual shortcut connection to match
                # changed dims
                x = resnet_layer(inputs=x,
                                 num_filters=num_filters_out,
                                 kernel_size=1,
                                 strides=strides,
                                 activation=None,
                                 batch_normalization=False)
            x = keras.layers.add([x, y])

        num_filters_in = num_filters_out

    # Add classifier on top.
    # v2 has BN-ReLU before Pooling
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    if dropout_flag == 1 or dropout_flag == 4 or dropout_flag == 5 or dropout_flag == 7:
        x = Dropout(dropout_value)(x)
    x = AveragePooling2D(pool_size=8)(x)
    y = Flatten()(x)
    if dropout_flag == 2 or dropout_flag == 4 or dropout_flag == 6 or dropout_flag == 7:
        y = Dropout(dropout_value)(y)
    outputs = Dense(num_classes,
                    activation='softmax',
                    kernel_initializer='he_normal')(y)
    if dropout_flag == 3 or dropout_flag == 5 or dropout_flag == 6 or dropout_flag == 7:
        outputs = Dropout(dropout_value)(outputs)
    return outputs
